In [9]:
using MLDatasets, Flux

train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :) # insert trivial channel dim
    yhot  = Flux.onehotbatch(data.targets, 0:9)  # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

# net = Chain(
#     Conv((3, 3), 1 => 6,  relu),
#     MaxPool((2, 2)),
#     Conv((3, 3), 6 => 16, relu),
#     MaxPool((2, 2)),
#     Flux.flatten,
#     Dense(400 => 84, relu), 
#     Dense(84 => 10, identity),
# )
# ew. prostsza architektura:
net = Chain(
    Conv((3, 3), 1 => 6,  relu, bias=false),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(13*13*6 => 84, relu, bias=false), 
    Dense(84 => 10, identity, bias=false)
)

Chain(
  Conv((3, 3), 1 => 6, relu, bias=false),  # 54 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(1014 => 84, relu; bias=false),  # 85_176 parameters
  Dense(84 => 10; bias=false),          # 840 parameters
)                   # Total: 3 arrays, 86_070 parameters, 336.820 KiB.

In [10]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    ŷ = model(x)
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [0 5]
loss_and_accuracy(net, test_data) = (loss = 2.325275f0, acc = 11.8, split = :test)


In [11]:
include("reverse_diff.jl")

for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        grads = custom_gradient(net, x, y)
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

# include("reverse_diff.jl")
# @info calculateGradient();

  4.661809 seconds (475.65 k allocations: 4.797 GiB, 5.57% gc time, 4.07% compilation time)


┌ Info: 1
│   acc = 89.91
│   test_acc = 90.88
└ @ Main /home/tomasz/projects/awid/cnn.ipynb:11


  4.835550 seconds (264.87 k allocations: 4.785 GiB, 12.94% gc time)


┌ Info: 2
│   acc = 91.77
│   test_acc = 92.68
└ @ Main /home/tomasz/projects/awid/cnn.ipynb:11


  4.843145 seconds (264.87 k allocations: 4.785 GiB, 12.88% gc time)


┌ Info: 3
│   acc = 92.82
│   test_acc = 93.28
└ @ Main /home/tomasz/projects/awid/cnn.ipynb:11


In [12]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, train_data);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [9 9]
loss_and_accuracy(net, train_data) = (loss = 0.24484529f0, acc = 92.82, split = :train)
